# 概要
- スクレイピングで対象ページの記事を全てdbに落とす
-  とりあえずdb (id, category, text, url)で定義する

# ライブラリimport 

In [1]:
import requests
import urllib
import sqlite3
import re
import time
from bs4 import BeautifulSoup as BS
from tqdm import tqdm_notebook as tqdm

# スクレイピング用の関数定義

In [2]:
# トップページからカテゴリ別に分けられてるページのURLを抽出
def main_url_func(main_url):
    main_url_dic = {}
    res = urllib.request.urlopen(main_url)
    data = res.read()
    text = data.decode("utf-8")
    soup = BS(text, 'html.parser')
    ul = soup.html.body.nav.ul
    li = ul.find_all('li', class_=re.compile(r'^(nav_color_).$'),)

    for i in li:    
        main_url_dic[i.a.string] = i.a.attrs['href']
    return main_url_dic


# カテゴリごとのページを訪れ、次のページへのリンクを抽出
def same_category_other_page_url(main_dic):

    for category, start_url  in main_dic.items():
        other_pages_list = [start_url]
        all_url_dic = {}
        
        url = start_url
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
    
        body = soup.html.body
        li = body.find_all("a", href=re.compile(r'^/categories/1.'))
    
        for i in li:
            other_pages_list.append("https://gunosy.com" + i.attrs['href'])
            
        all_url_dic[category] = set(other_pages_list)
        
    return all_url_dic


#　カテゴリ別のページにアクセスして記事のあるURLを抽出
def category_url_func(main_dic):
    sub_url_dic = {}
    
    for category, url in main_dic.items():
        url_list = []  
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
        soup_html = soup.html.body
        li = soup_html.find_all("a", href=re.compile(r'^https://gunosy.com/articles/'))
    
        for i in li:
            url_list.append(i.attrs['href'])
        
        sub_url_dic[category] = url_list
    
    return sub_url_dic

# 記事のtextをスクレイピング
#　URLを入力したら、textがリストで返る
def text_scraping(url):
    try:
        text_list = []
        res = urllib.request.urlopen(url)
        time.sleep(1)
        data = res.read()
        text = data.decode("utf-8")
        soup = BS(text, "html.parser")
        body = soup.html.body
        articlie = body.find_all("p")
        for i in articlie:
            text_list.append(i.string)
    except :
        print("ERROR")
        print("{}".format(url))
    return text_list

# URLの取得

In [3]:
top_url = "https://gunosy.com"
main_url_dic = main_url_func(top_url)id real, category real, airticle text, url text

category_url_dic = category_url_func(main_url_dic)

# dbの定義

In [4]:
# dbに接続する
conn = sqlite3.connect('GunosyText.db')
c = conn.cursor()

# テーブルの作成
try:
    c.execute('''CREATE TABLE textlist(id real, category real, airticle text, url text)''')
except:
    print("すでにDBがあります")

# 記事のスクレイピングと記事をdbに保存
**「逐次的にアクセスして保存」**
1. 記事のURLにアクセス
2. 記事をスクレイピング
3. 　dbに保存

In [5]:
count = 0
category_dic = {}
for name in category_url_dic.keys():
    category_dic[name] = count
    count += 1
category_dic

{'エンタメ': 0,
 'スポーツ': 1,
 'おもしろ': 2,
 '国内': 3,
 '海外': 4,
 'コラム': 5,
 'IT・科学': 6,
 'グルメ': 7}

## テスト用のコード

In [6]:
# (id, category, text, url)
""" 

index = 0

for name, url_list in {"IT・科学":category_url_dic['グルメ']}.items():
    
    for url in tqdm(url_list[50:]):
        text = text_scraping(url)
        text = "　".join(map(str, text))
        data = [(index, category_dic[name], text, url)]
        c.executemany("INSERT INTO textlist VALUES(?, ?, ?, ?)", data)
        index += 1
c.close()
""" 


' \n\nindex = 0\n\nfor name, url_list in {"IT・科学":category_url_dic[\'グルメ\']}.items():\n    \n    for url in tqdm(url_list[50:]):\n        text = text_scraping(url)\n        text = "\u3000".join(map(str, text))\n        data = [(index, category_dic[name], text, url)]\n        c.executemany("INSERT INTO textlist VALUES(?, ?, ?, ?)", data)\n        index += 1\nc.close()\n'

## 本ちゃんコード

In [15]:
# (id, category, text, url)
index = 0

for name, url_list in category_url_dic.items():
    
    for url in tqdm(url_list):
        text = text_scraping(url)
        text = "　".join(map(str, text))
        data = [(index, category_dic[name], text, url)]
        c.executemany("INSERT INTO textlist VALUES(?, ?, ?, ?)", data)
        index += 1
        # cではない・要注意
        conn.commit()
c.close()

ERROR
https://gunosy.com/articles/auM8y
ERROR
https://gunosy.com/articles/auM8y



# ERROR
- 存在しないページがあるっぽい
- 例外処理をしないといけない

# DBの確認

In [16]:
conn = sqlite3.connect('GunosyText.db')
c = conn.cursor()
c.execute("select * from textlist where category=?", (7,))
#c.execute("select * from textlist")
c.fetchall()

[(407.0,
  7.0,
  'グノシー\u3000しゃぶしゃぶを「鍋」と言っていいのか？\u3000よくよく考えると、難しいところだ。水炊きやカニ鍋、あんこう鍋であればド真ん中ストライクの「なべ〜!!」って感じだが、しゃぶしゃぶだと「なべ!?」ってなる人も多いのではないだろうか。\u3000None\u3000経費で落ちるからだ。\u3000正直なところ、そこは大きい。自腹だとしたら、1000円以上かかる鍋なんて絶対に紹介しない。というか、こんな企画自体やらない。家で普通に鍋食ってる。\u3000……というのが本音ではあるものの、経費でお店を訪れているうちに「次は自腹で行こうかな」と思うサービスに出会うことは珍しくない。その1つが、夢庵の「しゃぶしゃぶ食べ放題」だ。\u3000結論から先に言ってしまうと、「バーミヤンの火鍋しゃぶしゃぶ食べ放題」よりも夢庵の方が味の満足度は高かった。特にお肉。\u3000仕入れ状況等にもよるので一概には言えないが、私が訪れたときは「バーミヤンよりも やや美味しいかも!?」という印象。バーミヤンも、価格と味を考えれば満足度は相当に高かったのだが。\u3000なお、バーミヤンと夢庵は同じ系列（すかいらーくグループ）なので、「しゃぶしゃぶ食べ放題」で似ている点は非常に多い。使われている鍋とか、たぶん一緒だ。全店舗でどうかは分からないけれど、少なくとも私が訪れた店舗に限れば。\u3000・プランいろいろ\u3000さて、夢庵でしゃぶしゃぶ食べ放題を食べる場合、まずは「牛肉＆豚肉の食べ放題」にするか「豚肉の食べ放題」にするかを選ぶことになる。どちらを選んでも野菜等は食べ放題だが、簡単に言うと「ちょい高くなるのを覚悟で牛肉も食べ放題にするか」を決めるって感じだ。\u3000None\u3000参考までにホームページ上の価格をお伝えすると、牛・豚しゃぶコースのプレミアムプランが2480円で、スタンダードプランは1980円。豚しゃぶコースの場合、プレミアムプランが2180円で、スタンダードプランが1680円になる。\u3000なお、上の価格はすべて税抜きで、「大人」の場合。小学生やシニアになってくるとまた変わってくる上に、一部店舗では価格が異なるそうだ。\u3000そして食べ放題は100分。その時間を考えれば、1人で色々と食べるのは厳しいはず